In [ ]:
# -*- coding: utf-8 -*-
# vim: tabstop=4 shiftwidth=4 softtabstop=4
#
# BITE - A Basic/IoT/Example
# Copyright (C) 2020-2021 Daniele Viganò <daniele@vigano.me>
#
# BITE is free software: you can redistribute it and/or modify
# it under the terms of the GNU Affero General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# BITE is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Affero General Public License for more details.
#
# You should have received a copy of the GNU Affero General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

import requests
import pandas as pd
import ipywidgets as widgets
import plotly.graph_objects as go

from IPython.display import display

from datetime import date, timedelta

serial = widgets.Text(
    value='esp32_1',
    placeholder='Type something',
    description='Serial:',
    disabled=False
)
date_from = widgets.DatePicker(
    description='From Date',
    disabled=False,
    value=date.today()
)
date_to = widgets.DatePicker(
    description='To Date',
    disabled=False,
    value=date.today() + timedelta(days=1)
)
display(serial)
display(date_from)
display(date_to)

In [ ]:
try:
    r = requests.get("http://localhost/telemetry/{serial}/{date_from}/{date_to}/".format(
        serial=serial.value,
        date_from=date_from.value,
        date_to=date_to.value
    ))
    data = r.json()
except requests.exceptions.RequestException as e:
    raise SystemExit(e)

df = pd.json_normalize(data)
index = pd.to_datetime(df['time'])
df = df.set_index(index)
df = df.resample('5min').agg(['min', 'max', 'mean'])

In [ ]:
fig = go.Figure([go.Scatter(x=df.index, y=df['payload.wifi-rssi']['max'], showlegend=False),
                 go.Scatter(x=df.index, y=df['payload.wifi-rssi']['mean'], fill="tonexty", name="WiFI RSSI"),
                 go.Scatter(x=df.index, y=df['payload.wifi-rssi']['min'], fill="tonexty", showlegend=False),
                 go.Scatter(x=df.index, y=df['payload.hall']['max'], showlegend=False),
                 go.Scatter(x=df.index, y=df['payload.hall']['mean'],fill="tonexty", name="Hall effect"),
                 go.Scatter(x=df.index, y=df['payload.hall']['min'], fill="tonexty", showlegend=False)])
fig.show()